In [1]:
import numpy as np
import pandas as pd

In [2]:
movies= pd.read_csv('tmdb_5000_movies.csv')
credits= pd.read_csv('tmdb_5000_credits.csv')

In [3]:
movies = movies.merge(credits,on='title',how='inner')

In [4]:
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [5]:
import ast

In [6]:
def convert(text):
    L = []
    for i in ast.literal_eval(text):
        L.append(i['name']) 
    return L 

In [7]:
movies['genres']= movies['genres'].apply(convert)

In [8]:
movies['keywords']= movies['keywords'].apply(convert)

In [9]:
def convertcast(text):
    L = []
    for i in ast.literal_eval(text):
        L.append(i['name'])
        if len(L)> 4 : break
    return L 

In [10]:
movies['cast']= movies['cast'].apply(convertcast)

In [11]:
def director(text):
    L = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            L.append(i['name'])
            break
    return L 

In [12]:
movies['Director']= movies['crew'].apply(director)
# movies.head(4)

In [13]:
def producer(text):
    L = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Producer':
            L.append(i['name'])
            break
    return L 

In [14]:
movies['Producer']= movies['crew'].apply(producer)
# movies.head(4)

In [15]:
def writer(text):
    L = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Writer':
            L.append(i['name'])
            break
    return L 

In [16]:
movies['Writer']= movies['crew'].apply(producer)

In [17]:
movies.dropna(inplace=True)

In [18]:
movies['overview'] = movies['overview'].apply(lambda x: x.split())

In [19]:
def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
    return L1

In [20]:
movies['cast'] = movies['cast'].apply(collapse)
movies['crew'] = movies['crew'].apply(collapse)
movies['genres'] = movies['genres'].apply(collapse)
movies['keywords'] = movies['keywords'].apply(collapse)
movies['Director'] = movies['Director'].apply(collapse)
movies['Producer'] = movies['Producer'].apply(collapse)
movies['Writer'] = movies['Writer'].apply(collapse)

In [21]:
movies['tags']= movies['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['Director']+movies['Producer']+movies['Writer']

In [22]:
movies= movies[['movie_id','title','tags']]

In [48]:
movies['tags'] = movies['tags'].apply(lambda x: " ".join(x))

In [50]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=10000,stop_words='english')

In [60]:
vector = cv.fit_transform(movies['tags']).toarray()

In [56]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vector)

In [129]:
movie = movies[movies['title'] == 'The Lego Movie'].index[0]

In [201]:
def suggest(movie):
    index = movies[movies['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(movies.iloc[i[0]].title)

In [223]:
suggest('Harry Potter and the Half-Blood Prince')

Harry Potter and the Order of the Phoenix
Harry Potter and the Goblet of Fire
Harry Potter and the Chamber of Secrets
Harry Potter and the Philosopher's Stone
Harry Potter and the Prisoner of Azkaban


In [225]:
import pickle

In [231]:
pickle.dump(movies.to_dict(),open('movie_dict.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))